In [1]:
from climtools import climtools_lib as ctl
import spectfbcalc_lib as sfc
import output_lib as ol

No DISPLAY variable set. Switching to agg backend


In [2]:
from importlib import reload
reload(sfc)
reload(ol)

<module 'output_lib' from '/work/users/malbanese/radspesoft/SpectFbCalc/spectfbcalc/output_lib.py'>

In [3]:
sfc.mytestfunction()

test!


In [4]:
ctl.datestamp()

'2025-12-03T15:19:15'

In [5]:
import sys
import os
import glob
import re

import numpy as np
import xarray as xr
import pandas as pd

from climtools import climtools_lib as ctl
from matplotlib import pyplot as plt
import matplotlib.cbook as cbook
from scipy import stats
import pickle
import dask.array as da
import yaml
from difflib import get_close_matches
import re
import cartopy.crs as ccrs
import cartopy.feature as cfeature

## lib test

In [ ]:
config = "path/to/config/file/config.yaml"
if not os.path.exists(config):
    raise FileNotFoundError(
        f"Configuration file '{config}' not found! "
        "copy 'spectfbcalc/config_example.yaml' and personalize it."
    )
kernel = ""
standard_dict = "path/to/config/variable/file/configvariable.yaml"

In [ ]:
rad_palp = sfc.Rad_anomaly_planck_surf_wrapper(config, kernel, standard_dict)

In [ ]:
rad_psw = sfc.Rad_anomaly_planck_atm_lr_wrapper(config, kernel, standard_dict)

In [ ]:
rad_a = sfc.Rad_anomaly_albedo_wrapper(config, kernel, standard_dict)

In [ ]:
rad_wv = sfc.Rad_anomaly_wv_wrapper(config, kernel, standard_dict)

In [ ]:
all_anoms = sfc.calc_anoms_wrapper(config, kernel, standard_dict)

In [ ]:
fb_cloud = sfc.feedback_cloud_wrapper(config, kernel, standard_dict)

In [ ]:
fb_coef, fb_cloud, fb_cloud_err = sfc.calc_fb_wrapper(config, kernel, standard_dict)

In [ ]:
fb_coef_interannual = sfc.calc_fb_interannual_wrapper(config, kernel, standard_dict)

In [ ]:
feedback = sfc.calc_fb_wrapper(config, kernel, standard_dict)

In [ ]:
#to visualize feedback

fb_coef, fb_cloud, fb_cloud_err, fb_pattern, fb_cloud_pattern = feedback
print("Feedback Coefficients:")
for key, value in fb_coef.items():
    print(f"{key}: slope={value.slope:.4f}, stderr={value.stderr:.4f}")

# print(f"cloud: slope={fb_cloud:.4f}, stderr={fb_cloud_err:.4f}")
print("\nFeedback Patterns:")
for key, (slope, stderr) in fb_pattern.items():
    print(f"{key}: slope shape = {slope.shape}, stderr shape = {stderr.shape}")

# for key, (slope, stderr) in fb_cloud_pattern.items():
#     print(f"{key}: slope shape = {slope.shape}, stderr shape = {stderr.shape}")

In [ ]:
# output paths
out_txt = "path/to/output/feedback/values/file/feedback.txt"
out_nc = "path/to/output/feedback/patterns/file/feedback_patterns.nc"

# save results
ol.save_feedback_output(feedback, out_txt, out_nc)

#or save_feedback_output(output, out_txt)

In [ ]:
out_nc = "path/to/output/feedback/patterns/file/feedback_patterns.nc"
ds = xr.open_dataset(out_nc)

# Determine cold/warm from filename
filename = os.path.basename(out_nc).lower()  # e.g., 'feedback_patterns.nc' might not help
foldername = os.path.basename(os.path.dirname(os.path.dirname(out_nc)))  # e.g., 'pi9r'

if foldername.endswith("9r"):
    run_label = "cold"
elif foldername.endswith("c9"):
    run_label = "warm"
elif foldername == "c4r8":
    run_label = "control"
else:
    run_label = "unknown"

for varname in ds.data_vars:
    if varname.endswith("_slope"):
        slope = ds[varname]
        stderr = ds[varname.replace("_slope", "_stderr")]

        cloud_type, component = varname.replace("_slope", "").split("_", 1)
        feedback_name = component.capitalize()
        title = f"{feedback_name} ({cloud_type}) - {run_label}"

        ol.plot_fb_pattern(
            slope = slope,
            stderr = stderr,
            title = title,
            output_folder = "path/to/output/plot/folder",
            filename_prefix = f"fb_pattern_{cloud_type}_{component}"
        )

In [ ]:
import xarray as xr

ds = xr.open_dataset("path/to/feedback/patterns/file/feedback_patterns.nc")
output_folder = "path/to/output/plot/folder"

ol.save_all_fb_patterns_to_pdf(
    ds=ds,
    output_folder=output_folder,
    pdf_name="fb_patterns_c49r.pdf", #example
    plot_function=ol.plot_fb_pattern,
    run_label="cold"   # 👈 specify experiment here
)

In [ ]:
#plot simulation slopes in a gregory plot with 95 % CI
ol.plot_single_feedback_file(
    feedback_file="path/to/feedback/values/file/feedback.txt",
    sim_label="tunecs_c4c9",
    save_path="path/to/output/plot/folder/plot_name.png"
)

In [ ]:
# evaluation tuning parameter change (related to a single feedback) effect on dRt 

# param_map_ece3 = {
#             ...
#         }
param_map_ece4 = {
            ...
}

base_folder = "/path/to/base/folder/with/computed/dRt/values/"
xlabel = ""
ylabel = "dRt (W/m²)"
title = "ECE4 - HUANG - Water Vapor Feedback - 20% variation" #example
output_path = "/path/to/output/plot/folder/plot_name.png"
feedback_file = "dRt_water-vapor_global_clr_climatology-HUANGkernels.nc"
subfolder_pattern = ""
model = "" # ece3 or ece4

df = ol.plot_dRt_fb_all_params( base_folder, xlabel, ylabel, title, feedback_file, output_file=output_path, subfolder_pattern=subfolder_pattern, param_map=param_map_ece4, model_type=model)

In [ ]:
#plotting dRt and NetToa time series

base_folder = "/path/to/base/folder/with/computed/dRt/values/"
par = "" # param has been changed
dRt = "/path/to/folder/with/dRt/time_s/values/"
title = "Clr Net TOA Anomaly & dRt Components for RCLDIFF_CONVI variation 20 % (HUANG) - timeseries" #example
output = "/path/to/output/plot/folder/plot_name.png"

ol.plot_toa_anomaly(base_folder, param=par, dRt_folder=dRt, title=title, model_type="ece4", sky="clr", output_file=output)